https://www.kaggle.com/khanrahim/fake-news-classification-easiest-99-accuracy

In [6]:
# !pip install pandas
# !pip install matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

Importing dataset

In [7]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Data preprocessing and cleaning

In [9]:
#filling NULL values with empty string
df = df.fillna('')
test = test.fillna('')

In [10]:
# We will be only using title and author name for prediction
# Creating new coolumn total concatenating title and author
df['total'] = df['title'] + ' ' + df['author']
test['total'] = test['title'] + ' ' + test['author']

In [12]:
X = df.drop('label', axis=1)
y = df['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [33]:
#Choosing vocabulary size to be 5000 and copying data to msg for further cleaning
voc_size = 5000
msg = X.copy()
msg_test = test.copy()

In [34]:
# !pip install nltk
#Downloading stopwords 
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\smarotta.EXPERTS
[nltk_data]     YSTEM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
#Stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [ ]:
#Applying stemming and some preprocessing
# print(msg)
for i in range(len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['total'][i])
    review = review.lower()
    review = review.split()
    print(len(review))
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# print(corpus[1])

18
11
9
11
16
21
17
17
21
21
17
11
15
11
15
16
15
14
7
12
23
12
9
15
22
13
19
13
17
12
13
24
14
18
25
15
13
7
12
19
12
12
15
14
14
21
16
0
15
19
6
7
16
1
14
11
14
10
18
13
15
15
20
12
19
16
14
14
14
16
9
9
14
15
15
15
20
11
15
13
20
15
7
11
22
21
3
16
21
11
18
17
18
25
8
12
8
13
14
21
14
13
18
13
10
14
15
11
13
15
20
15
18
20
17
17
17
15
14
22
1
16
18
18
1
18
14
16
14
15
15
10
18
21
11
15
14
14
8
23
1
16
15
17
16
13
17
14
13
11
18
17
15
13
17
15
6
20
14
9
21
17
13
9
13
22
15
13
11
13
8
13
10
13
25
7
21
14
19
8
15
18
16
22
11
10
8
14
13
20
17
11
17
11
8
14
1
20
11
23
21
12
11
19
5
19
11
8
14
5
16
20
14
13
10
8
13
20
15
19
16
10
18
1
19
16
14
15
15
0
15
7
14
20
19
8
9
13
16
14
14
20
22
16
12
19
14
16
6
18
9
20
14
16
18
18
13
6
15
18
12
17
14
18
17
18
14
9
19
14
26
20
13
18
17
11
5
16
20
18
15
19
10
7
19
8
21
16
1
15
14
14
16
18
19
13
18
15
10
13
16
13
18
14
15
15
12
11
17
13
12
16
16
17
16
13
21
61
5
17
19
18
8
12
9
15
19
28
19
16
21
13
10
7
10
16
11
13
7
21
9
18
12
7
14
6
12
2
6
17
10
1

15
12
0
9
12
12
18
11
12
17
16
2
15
12
13
12
11
14
14
17
12
13
11
17
14
16
19
12
23
27
12
17
16
4
10
15
8
25
4
8
13
8
1
14
18
16
20
20
35
13
13
13
13
12
18
15
17
15
7
17
8
15
16
18
2
22
19
16
15
10
12
13
18
13
13
12
12
19
18
14
14
14
13
13
17
12
13
16
10
14
13
11
21
11
14
17
20
15
18
6
17
20
13
5
16
12
9
16
13
7
16
12
15
14
14
18
9
8
17
9
16
13
12
18
14
8
13
9
16
19
18
20
15
20
11
14
20
8
17
10
5
20
10
18
10
11
16
13
15
21
12
13
18
10
9
13
16
12
12
14
19
16
12
12
9
18
13
15
15
12
13
10
14
17
24
23
15
9
20
9
12
14
13
1
12
14
16
18
17
18
17
10
9
20
9
12
9
17
11
12
20
17
17
22
14
15
13
12
10
11
13
8
11
8
14
13
15
22
16
18
37
15
18
8
14
10
16
10
13
16
11
15
12
7
20
12
14
13
15
16
15
16
23
16
7
19
0
13
17
21
19
20
22
16
17
14
17
18
7
16
14
12
15
11
17
18
9
13
12
10
12
20
16
18
15
14
14
9
1
10
14
0
14
3
16
14
11
39
16
12
15
13
10
17
6
10
12
11
14
11
11
17
14
11
4
19
13
18
13
13
12
18
20
24
11
10
13
11
12
23
10
5
7
8
23
13
13
17
24
13
13
13
8
12
16
14
11
16
20
18
12
16
15
18
15
2
18
15
10
12


12
13
7
15
17
8
16
40
15
17
6
17
19
7
15
19
13
15
7
17
18
0
21
14
16
13
10
14
13
15
11
25
20
4
22
16
13
15
11
8
17
9
22
8
14
21
19
13
21
16
11
13
12
18
10
9
19
20
16
13
4
6
11
13
21
15
18
16
23
12
16
14
12
14
17
20
11
16
19
12
12
11
15
14
16
23
8
15
11
5
15
12
8
19
9
8
16
12
22
12
21
2
11
23
18
18
18
7
15
4
19
10
17
12
17
21
18
6
11
16
13
25
15
6
16
19
10
16
17
7
18
18
21
14
10
16
15
19
14
16
16
16
22
9
15
12
16
2
18
19
14
15
12
16
13
10
18
9
18
18
13
15
7
8
8
19
43
18
11
22
15
18
11
23
15
22
12
17
15
13
11
9
52
17
16
17
15
15
17
18
10
19
3
14
11
8
8
13
18
9
12
15
17
14
10
8
13
10
8
16
13
14
16
11
0
17
17
12
16
1
15
5
15
12
21
16
16
14
14
20
17
15
25
5
15
7
15
17
17
18
6
16
13
18
22
12
18
17
13
18
17
14
0
22
13
15
12
16
16
14
25
12
11
15
17
6
16
8
12
19
17
15
11
15
23
18
15
9
16
19
19
15
20
22
21
15
11
16
8
18
14
13
12
17
10
15
20
15
16
18
17
10
13
19
15
16
19
15
14
16
20
19
15
17
11
25
18
9
12
11
11
10
13
18
22
19
13
20
17
20
13
16
28
8
10
19
17
12
9
16
17
19
13
11
16
11
17
9
15
15
16